# Decays

What we want to try to do in this notebook is to get the correct expression for the decay of the Muon without any fancy loop corrections. This exercise is one that can be found in Griffiths. We want to try to do it with SymPy.

## (#1): Libraries:

In [1]:
import sympy as sp

## (#2): Constants:

### (#2.1): Masses:

In [9]:
# LEPTONS
mass_electron_in_GeV = sp.symbols('m_e', real = True)
mass_muon_in_GeV = sp.symbols('m_mu', real = True)
mass_tau_in_GeV = sp.symbols('m_tau', real = True)

mass_electron_neutrino_in_GeV = sp.symbols('m_nu_e', real = True)
mass_muon_neutrino_in_GeV = sp.symbols('m_nu_mu', real = True)
mass_tau_neutrino_in_GeV = sp.symbols('m_nu_tau', real = True)

# QUARKS:
mass_quark_up_in_GeV = sp.symbols('m_u', real = True)
mass_quark_down_in_GeV = sp.symbols('m_d', real = True)
mass_quark_strange_in_GeV = sp.symbols('m_s', real = True)
mass_quark_charm_in_GeV = sp.symbols('m_c', real = True)
mass_quark_bottom_in_GeV = sp.symbols('m_b', real = True)
mass_quark_top_in_GeV = sp.symbols('m_t', real = True)

# GAUGE BOSONS:
mass_W_boson_in_GeV = sp.symbols('m_W', real = True)

### (#2.2): Couplings:

In [10]:
# COUPLINGS:
gW_in_GeV = sp.symbols('g_W', real = True)

### (#2.3): Symbolic Variables:

In [11]:
# VARIABLES:
m23Squared = sp.Symbol("m_23^2", real = True)
m12Squared = sp.Symbol("m_12^2", real = True)

## (#3): Two-Body Decay Function:

Two-body decays are easy because there are enough constraints in the problem that no integrations are necessary. The equation that we need to code is just

$$\Gamma \left( A \to 1 + 2 \right) = \frac{S |\mathcal{M}|^{2}}{8 \pi m_{A}^{2}} \frac{1}{2 m_{A}} \sqrt{m_{A}^{4} + m_{1}^{4} + m_{2}^{4} - 2 \left( m_{A}^{2} m_{1}^{2} + m_{A}^{2} m_{2}^{2} + m_{1}^{2} m_{2}^{2}\right)}.$$

For some more clarity, $S$ is a "spin factor" that is necessary if the two final state particles are identical (e.g. two electrons), and the fancy, huge square root term is called the $\textbf{Källén Function}$, defined as

$$\lambda(x, y, z) := x^{2} + y^{2} + z^{2} - 2 x y - 2 x z - 2 y z.$$

Of course, $\mathcal{M}$ is the amplitude of the process that we calculate within QFT (Quantum Field Theory).

### (#3.1): The Källén Function:

In [12]:
def kallen_function(x, y, z):

    try:
        kallen_polynomial = sp.Pow(x, 2) + sp.Pow(y, 2) + sp.Pow(z, 2) - 2 * (x * y + y * z + x * z)
        return kallen_polynomial
    except:
        print(f">Error!")
        return 0.

### (#3.2): Two-Body Decay Function:

In [13]:
def decay_two_body(amplitude_squared, mass_A, mass_1,  mass_2, spin_factor: int = 1):
    
    try:

        # (1): The Spin Factor:
        spin_factor = spin_factor

        # (2): Calculate the Two-Body Decay Prefactor:
        two_body_decay_prefactor = spin_factor / (16 * sp.pi * sp.Pow(mass_A, 3))

        # (3): Evaluate the Kallen Polynomial:
        kallen_polynomial = sp.sqrt(kallen_function(sp.Pow(mass_A, 2), sp.Pow(mass_1, 2), sp.Pow(mass_2, 2)))

        # (4): Return the actual expression:
        two_body_decay_rate = two_body_decay_prefactor * kallen_polynomial * amplitude_squared
        
        # (5): Return the expression:
        return two_body_decay_rate
    
    except Exception as ERROR:
        print(f"> Error calculating two-body decay rate:\n {ERROR}")
        return 0

### (#3.3): Examples of Two-Body Decay:

#### (#3.3.1): Amplitude Squared of Unity:

In [14]:
decay_two_body(1, mass_muon_in_GeV, 0,  0, 1)


1/(16*pi*m_mu)

#### (#3.3.2): Amplitude Squared of Unity with Some Nonzero Final-State Masses:

In [18]:
decay_two_body(16 * 5, mass_muon_in_GeV, mass_tau_in_GeV, mass_electron_in_GeV)

5*sqrt(m_e**4 - 2*m_e**2*m_mu**2 - 2*m_e**2*m_tau**2 + m_mu**4 - 2*m_mu**2*m_tau**2 + m_tau**4)/(pi*m_mu**3)

## (#4): Three-Body Decay Function:

We'll now implement the "Dalitz Integration" method.

In [ ]:
def calculate_m23Squared_bounds(
        mass_mother, 
        mass_one, 
        mass_two,
        mass_three):

    e2Star = m12Squared - sp.Pow(mass_one, 2) + sp.Pow(mass_two, 2) / (2 * sp.sqrt(m12Squared)) 
    e3Star = sp.Pow(mass_mother, 2) - m12Squared - sp.Pow(mass_three, 2) / (2 * sp.sqrt(m12Squared))

    root_with_e2Star = sp.sqrt(sp.Pow(e2Star, 2) - sp.Pow(mass_two, 2))
    root_with_e3Star = sp.sqrt(sp.Pow(e3Star, 2) - sp.Pow(mass_three, 2))

    m23Squared_lower_bound = sp.Pow(e2Star + e3Star, 2) - sp.Pow(root_with_e2Star + root_with_e3Star, 2)
    m23Squared_upper_bound = sp.Pow(e2Star + e3Star, 2) + sp.Pow(root_with_e2Star + root_with_e3Star, 2)

    return (m23Squared_lower_bound, m23Squared_upper_bound)

def calculate_m12Squared_bounds(
        mass_mother, 
        mass_one, 
        mass_two,
        mass_three):
    
    m12Squared_lower_bound = sp.Pow(mass_one + mass_two, 2)
    m12Squared_upper_bound = sp.Pow(mass_mother - mass_three, 2)

    return (m12Squared_lower_bound, m12Squared_upper_bound)
    

def decay_three_body(
        mass_mother, 
        mass_one, 
        mass_two,
        mass_three,
        amplitude_squared):
    
    try:
        
        integrand = amplitude_squared / (sp.Pow(2 * sp.pi, 3) * 32 * sp.Pow(mass_mother, 3))

        m23SquaredLowerBound, m23SquaredUpperBound = calculate_m23Squared_bounds(mass_mother, mass_one, mass_two, mass_three)
        m12SquaredLowerBound, m12SquaredUpperBound = calculate_m12Squared_bounds(mass_mother, mass_one, mass_two, mass_three)

        dalitz_integration = sp.integrate(integrand, (m23Squared, m23SquaredLowerBound, m23SquaredUpperBound),  (m12Squared, m12SquaredLowerBound, m12SquaredUpperBound))

        return dalitz_integration

    except:
        print(f"> Error!")
        return 0